# Auto Polling
    - This notebook will serve the following pipeline
    - Ingest a demographic data, like the census of an area
        - Create groups that will in total represent the population
            - IE 100/1000 white women ages 20-30
        - Turn the groups into personas
            - For each group, use chat gpt to make that many personas of people
            - Save personas for later use
    - Create a survey based on the political landscape of the area
        - Feed in the data and make survey
        - Could make multiple surveys and have them be specific to the demographic
    - Give survey to the personas
        - For each persona, have them take the survey, save results
    - Process results to determine the most important issues to make ads for
        

In [7]:
import Utils.Chat_GPT_Funcs as GPT
import concurrent.futures
import os
import string

In [8]:
alphabet_uppercase = list(string.ascii_uppercase)

male_name_starts = ['Al', 'An', 'Be', 'Br', 'Ch', 'Da', 'De', 'Ed', 'Fr', 'Ge', 'Ha', 'Ja', 'Jo', 'Ke', 'Le', 'Ma', 'Mi', 'Ni', 'Pa', 'Ra', 'Ro', 'Sa', 'St', 'Th', 'To', 'Wi', 'Ba', 'Ca', 'Di', 'El', 'Gr', 'He', 'Ir', 'Je', 'Kr', 'Lu', 'Mo', 'Ne', 'Os', 'Ph', 'Qu', 'Ri', 'Se', 'Te', 'Va', 'Ze']
female_name_starts = ['Ab', 'Al', 'Am', 'An', 'Be', 'Br', 'Ca', 'Ce', 'Ch', 'Cl', 'De', 'Di', 'El', 'Em', 'Es', 'Ev', 'Fl', 'Fr', 'Ge', 'Gr', 'Ha', 'He', 'Ir', 'Is', 'Ja', 'Je', 'Jo', 'Ju', 'Ka', 'Ke', 'La', 'Li', 'Ma', 'Me', 'Mi', 'Na', 'Ni', 'Ol', 'Pa', 'Pr', 'Ra', 'Re', 'Sa', 'Si', 'Ta', 'Vi']

male_names = alphabet_uppercase + male_name_starts
female_names = alphabet_uppercase + female_name_starts

print(len(female_names))

72


In [9]:
test = [
  {
    "group_size": 294,
    "age": "18-64",
    "gender": "Female",
    "ethnicity": "White",
    "income_levels": "$20,000 - $50,000",
    "employment_status": "In civilian labor force (62.9%)",
    "marital_status": "Not specified",
    "education_level": "High school graduate or higher (94.4%)"
  },
  {
    "group_size": 146,
    "age": "65 and over",
    "gender": "Female",
    "ethnicity": "White",
    "income_levels": "$20,000 - $50,000",
    "employment_status": "Not in civilian labor force",
    "marital_status": "Not specified",
    "education_level": "High school graduate or higher (94.4%)"
  },
  {
    "group_size": 294,
    "age": "18-64",
    "gender": "Male",
    "ethnicity": "White",
    "income_levels": "$20,000 - $50,000",
    "employment_status": "In civilian labor force (62.9%)",
    "marital_status": "Not specified",
    "education_level": "High school graduate or higher (94.4%)"
  },
  {
    "group_size": 146,
    "age": "65 and over",
    "gender": "Male",
    "ethnicity": "White",
    "income_levels": "$20,000 - $50,000",
    "employment_status": "Not in civilian labor force",
    "marital_status": "Not specified",
    "education_level": "High school graduate or higher (94.4%)"
  },
  {
    "group_size": 20,
    "age": "18 and over",
    "gender": "Not specified",
    "ethnicity": "Black or African American",
    "income_levels": "$20,000 - $50,000",
    "employment_status": "Not specified",
    "marital_status": "Not specified",
    "education_level": "High school graduate or higher (94.4%)"
  },
  {
    "group_size": 66,
    "age": "18 and over",
    "gender": "Not specified",
    "ethnicity": "American Indian and Alaska Native",
    "income_levels": "$20,000 - $50,000",
    "employment_status": "Not specified",
    "marital_status": "Not specified",
    "education_level": "High school graduate or higher (94.4%)"
  },
  {
    "group_size": 10,
    "age": "18 and over",
    "gender": "Not specified",
    "ethnicity": "Asian",
    "income_levels": "$20,000 - $50,000",
    "employment_status": "Not specified",
    "marital_status": "Not specified",
    "education_level": "High school graduate or higher (94.4%)"
  },
  {
    "group_size": 1,
    "age": "18 and over",
    "gender": "Not specified",
    "ethnicity": "Native Hawaiian and Other Pacific Islander",
    "income_levels": "$20,000 - $50,000",
    "employment_status": "Not specified",
    "marital_status": "Not specified",
    "education_level": "High school graduate or higher (94.4%)"
  },
  {
    "group_size": 30,
    "age": "18 and over",
    "gender": "Not specified",
    "ethnicity": "Two or More Races",
    "income_levels": "$20,000 - $50,000",
    "employment_status": "Not specified",
    "marital_status": "Not specified",
    "education_level": "High school graduate or higher (94.4%)"
  },
  {
    "group_size": 43,
    "age": "18 and over",
    "gender": "Not specified",
    "ethnicity": "Hispanic or Latino",
    "income_levels": "$20,000 - $50,000",
    "employment_status": "Not specified",
    "marital_status": "Not specified",
    "education_level": "High school graduate or higher (94.4%)"
  }
]


In [10]:
test = [
    {
    "group_size": 146,
    "age": "65 and over",
    "gender": "Male",
    "ethnicity": "White",
    "income_levels": "$20,000 - $50,000",
    "employment_status": "Not in civilian labor force",
    "marital_status": "Not specified",
    "education_level": "High school graduate or higher (94.4%)"
  }
]

In [11]:
def create_directory_if_not_exists(file_path):
    directory = os.path.dirname(file_path)
    if not os.path.exists(directory):
        os.makedirs(directory)

In [12]:
def generate_and_save_persona(i, group_count):
  
    persona_gen_role = GPT.open_file("Prompts/persona_gen_groups_v1.txt")
    
    if group_count < 2:
        name_rep = female_names[i%len(female_names)]
    else:
        name_rep = male_names[i%len(male_names)]
        
    tags = {
        '<<BATCH>>': str(batch_size),
        '<<GROUP>>': str(original_dict),
        '<<START>>': name_rep,
        '<<NUM>>': str(i),
    }

    persona_text = GPT.generalized_gpt_prompt("Prompts/gen_by_group.txt", tags, tokens = 1500, engine='gpt-3.5-turbo', temp=0.25, role=persona_gen_role)
    
    parent_path = f"Groups_v2/Group_{group_count}/"
    create_directory_if_not_exists(parent_path)
    
    file_path = parent_path + f"group_{group_count}_batch_{i}.txt"
    print("Saving:", file_path)
    GPT.save_file(file_path, persona_text)

In [13]:
len(test)

1

In [ ]:
group_count = 3
for original_dict in test:
    
    print("Starting Group:", group_count)
    group_size = original_dict['group_size']

    # Remove the 'group_size' key from the new dictionary
    del original_dict['group_size']
    original_dict['place_of_residence'] = 'Montana, United States'

    import concurrent.futures

    batch_size = 5
    max_workers = 5

    # Calculate the number of full batches
    full_batches = group_size // batch_size

    # Calculate the number of remaining objects
    remaining_objects = group_size % batch_size

    personas = []

    # Create a ThreadPoolExecutor with max_workers
    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        # Create full batches of objects
        for i in range(full_batches):
            executor.submit(generate_and_save_persona, i, group_count)
            print("Starting batch:", str(i))
        # Create remaining objects
        if remaining_objects > 0:
            executor.submit(generate_and_save_persona, full_batches, group_count)
            print("Starting batch:", str(full_batches))
    
    group_count += 1
    print()
    print()


Starting Group: 3
Starting batch: 0
Starting batch: 1
Starting batch: 2
Starting batch: 3
Starting batch: 4
Starting batch: 5
Starting batch: 6
Starting batch: 7
Starting batch: 8
Starting batch: 9
Starting batch: 10
Starting batch: 11
Starting batch: 12
Starting batch: 13
Starting batch: 14
Starting batch: 15
Starting batch: 16
Starting batch: 17
Starting batch: 18
Starting batch: 19
Starting batch: 20
Starting batch: 21
Starting batch: 22
Starting batch: 23
Starting batch: 24
Starting batch: 25
Starting batch: 26
Starting batch: 27
Starting batch: 28
Starting batch: 29
Saving: Groups_v2/Group_3/group_3_batch_2.txt
Saving: Groups_v2/Group_3/group_3_batch_1.txt
Saving: Groups_v2/Group_3/group_3_batch_3.txt
Saving: Groups_v2/Group_3/group_3_batch_0.txt
Saving: Groups_v2/Group_3/group_3_batch_4.txt
Saving: Groups_v2/Group_3/group_3_batch_9.txt
Saving: Groups_v2/Group_3/group_3_batch_6.txt
Saving: Groups_v2/Group_3/group_3_batch_5.txt
Saving: Groups_v2/Group_3/group_3_batch_7.txt
Saving: 

In [22]:
def dict_to_paragraph(persona, residence):
    priorities = ', '.join(persona["Top Priorities"])
    try:
        pain_points = ', '.join(persona["Pain points"])
    except:
        pain_points = ', '.join(persona["Pain Points"])
    
    return (f"Your name is {persona['Name']}, you are a {persona['Age']} year old {persona['Ethnicity']} {persona['Gender']}. "
            f"You live in {residence} and your annual income is ${persona['Income']}. "
            f"Your marital status is {persona['Marital Status']}, and your highest level of education is {persona['Education Level']}. "
            f"You work as a {persona['Occupation']}. You are described as '{persona['Description']}'. "
            f"Your top priorities are {priorities}. You are facing some challenges, including {pain_points}.")


In [38]:
# import os
# import glob

# parent_path = "Groups/"
# # Define the directories to go through
# directories = ['Groups/Group_1', 'Groups/Group_2', 'Groups/Group_3', 'Groups/Group_4']

# # Define the string to find and the string to replace it with
# find_string = '"Nationality": "American",'
# find_string1 = '"Nationality": "White",'
# replace_string = '"Ethnicity": "White",'

# # For each directory
# for directory in directories:
#     # Use a wildcard (*) to find all .txt files in the directory
#     for file_name in glob.glob(directory + '/*.txt'):
#         print(file_name)
#         # Open each file in read mode to get the data
#         file_data = GPT.open_file(file_name)
        
#         # Replace the find_string with replace_string
#         file_data = file_data.replace(find_string, replace_string)
#         file_data = file_data.replace(find_string1, replace_string)
        
#         #print(new_data)
        
#         # Open the file in write mode to overwrite the original data with the new data
#         GPT.save_file(file_name, file_data)

# print("Replacement complete.")



In [23]:
import os
import glob
import json
parent_path = "Groups/"
# Define the directories to go through
directories = ['Groups/Group_1', 'Groups/Group_2', 'Groups/Group_3', 'Groups/Group_4']

personas = []
for i in range(0,11):
    group_p = []
    # For each directory
    # for directory in directories:

    directory = f'Groups/Group_{i}'
    # Use a wildcard (*) to find all .txt files in the directory
    for file_name in glob.glob(directory + '/*.txt'):
        # Open each file in read mode to get the data
        file_data = GPT.open_file(file_name)

        for data in json.loads(file_data):
#         for key, value in data.items():
#             print(f"{key}: {value}")
            personas.append(dict_to_paragraph(data, 'Montana, United States'))
            group_p.append(dict_to_paragraph(data, 'Montana, United States'))
    print(f"Num Personas in Group {i}:", len(group_p))





Num Personas in Group 0: 295

Num Personas in Group 1: 150

Num Personas in Group 2: 295

Num Personas in Group 3: 0

Num Personas in Group 4: 145

Num Personas in Group 5: 20

Num Personas in Group 6: 70

Num Personas in Group 7: 10

Num Personas in Group 8: 5

Num Personas in Group 9: 30

Num Personas in Group 10: 45



In [24]:
len(personas)

1065

In [25]:
personas[0]

"Your name is Nancy Nichols, you are a 32 year old White Female. You live in Montana, United States and your annual income is $35000. Your marital status is Single, and your highest level of education is High school graduate. You work as a Retail Sales Associate. You are described as 'Nancy is a friendly and outgoing individual who enjoys working with people. She is passionate about helping customers find the perfect items and providing excellent customer service.'. Your top priorities are Affordable healthcare, Job security, Environmental protection. You are facing some challenges, including Limited job opportunities in her area, Rising cost of living, Lack of affordable healthcare options."